# An Introduction to Using PyBEN

This is a small tutorial that is meant to help users get to using PyBEN: the python interface
for the [binary-ensemble](https://crates.io/crates/binary-ensemble) Rust package.


BEN (short for Binary-Ensemble) is a compression algorithm designed for efficient storage and
access of ensembles of districting plans, and was designed to work primarily as a companion to
the GerrySuite collection of packages (GerryChain, GerryTools, FRCW) and to also be compatible
with other ensemble generators (e.g. ForestRecom, Sequential Monte Carlo \[SMC\]). 

When working with an ensemble of plans, there is generally an underlying dual graph, $G$, and 
on which there is an ordering of nodes $(n_1, n_2, \ldots, n_\ell)$. If we then wish to 
partition the graph into, say, districts, then the only thing that we need to do is assign each
node in a graph to a district number. This is what we call the ***assignment vector*** for the 
districting plan. Then to encode an ensemble of districting plans in a JSONL file (short for JSON 
Lines and it really just means a file with a dictionary on every line), we may format each of the
lines in the following way:

```json
{"assignment": <assignment_vector>, "sample": <sample_number_indexed_from_1>}
```

However, if the graph has a lot of nodes in it and we want to collect millions of samples (as we 
tend to want to do), then this JSONL format can make for MASSIVE (tens or hundreds of Gb) files. So
this is why we have BEN and XBEN (e\[X\]treme BEN): to make the storage and processing of these
millions of plans possible without needing to buy an extra hard drive for every project that you 
would like to work with.

## Setup for the Tutorial

For this tutorial, you will need access to a few files. We are going to go ahead and download
them here and then place them in a folder called "example_data"

In [1]:
from urllib.request import urlopen
from pathlib import Path
import shutil

if Path("example_data").exists():
    shutil.rmtree("example_data")

Path("example_data").mkdir()

In [2]:
def open_and_save(base_url, file_name):
    url = f"{base_url}/{file_name}"
    out_path = f"./example_data/{file_name}"

    chunk = 1024 * 64
    with urlopen(url, timeout=120) as resp, open(out_path, "wb") as f:
        while True:
            buf = resp.read(chunk)
            if not buf:
                break
            f.write(buf)

url_base = "https://raw.githubusercontent.com/peterrrock2/binary-ensemble/main/example"
for file_name in [
    "CO_small.json",
    "small_example.jsonl",
]:
    out_path = f"./example_data/{file_name}"
    if not Path(out_path).exists():
        print(f"Downloading {file_name}...")
        open_and_save(url_base, file_name)
    else:
        print(f"{file_name} already exists, skipping download.")


url_base = "https://github.com/peterrrock2/binary-ensemble/raw/refs/heads/main/example/"
for file_name in [
    "100k_CO_chain.jsonl.xben",
]:
    out_path = f"./example_data/{file_name}"
    if not Path(out_path).exists():
        print(f"Downloading {file_name}...")
        open_and_save(url_base, file_name)
    else:
        print(f"{file_name} already exists, skipping download.")


url_base = "https://raw.githubusercontent.com/mggg/GerryChain/refs/heads/main/docs/_static"
for file_name in [
    "gerrymandria.json",
]:
    out_path = f"./example_data/{file_name}"
    if not Path(out_path).exists():
        print(f"Downloading {file_name}...")
        open_and_save(url_base, file_name)
    else:
        print(f"{file_name} already exists, skipping download.")

## Converting between file types

PyBen comes equiped with some utility functions for users who wish to convert between different
file types.

In [3]:
from pyben import (
    compress_jsonl_to_ben, compress_jsonl_to_xben, compress_ben_to_xben, decompress_ben_to_jsonl, decompress_xben_to_jsonl, decompress_xben_to_ben
)

### BEN compression

The most basic (and quickest) type of compression available is the BEN compression format. You 
may convert between a standard JSONL file to a BEN file using the following function:


In [4]:
compress_jsonl_to_ben(
    in_file="example_data/small_example.jsonl", 
    out_file="example_data/small_example_jsonl_to_ben.jsonl.ben",
)

As a small note, the above function (and all the conversion functions) has a default behavior of 
not overwriting output. 

In [5]:
try:
    compress_jsonl_to_ben(
        in_file="example_data/small_example.jsonl", 
        out_file="example_data/small_example_jsonl_to_ben.jsonl.ben",
    )
except OSError as e:
    print(f"Found Error: {e}")

Found Error: Output file example_data/small_example_jsonl_to_ben.jsonl.ben already exists (use overwrite=True to replace).


In addition, there is a `variant`
parameter with two options: "standard" and "mkv_chain". The "mkv_chain" variation is a special 
version of BEN that is optimized for ensembles generated using an MCMC method with a non-zero 
rejection probability (so the generated maps may repeat a few times to target an appropriate 
probability distribution like in [Reversible ReCom](https://mggg.org/rrc)).

For perfectly random ensembles, the output size of the "mkv_chain" variant is very slightly larger
than the "standard" variant, but for MCMC chains, the savings can be significant, so "mkv_chain"
is set as the default variant.

### XBEN Compression

XBEN (short for e\[X\]treme BEN) is a much more powerful version of our compression. In fact, with
some coercing of the data, it is not uncommon to get 1000x compression compared to base JSONL files.
However, all of these savings come at a cost: time and compute power. In general, while XBEN is 
relatively quick to decompress, it can take up to a few hours to compress a large sample. So this
format is great for when the user wants to store data long-term, but is less good in an actively 
changing project.  

In [6]:
compress_jsonl_to_xben(
    in_file="example_data/small_example.jsonl", 
    out_file="example_data/small_example_jsonl_to_xben.jsonl.xben",
    overwrite=True,
    variant="mkv_chain",
    n_threads=1,
    compression_level=9,
)

compress_ben_to_xben(
    in_file="example_data/small_example_jsonl_to_ben.jsonl.ben", 
    out_file="example_data/small_example_jsonl_to_ben_to_xben.jsonl.xben",
    overwrite=True,
    n_threads=1,
    compression_level=9,
)

There are now a few new parameters added to the XBEN compression functions: `n_threads` and 
`compression_level`. 

- `n_threads`: In the interest of actually finishing the compression at a reasonable 
paste, XBEN has been parallelized to allow the user to take advantage of modern CPUs with 
higher thread counts. So increasing the number of threads in the parameter will decrease the 
compression time. 

- `compression_level`: There are 10 possible compression levels 0 (fastest) - 9 (slowest) (these
follow the XZ compression levels). The higher the compression level, the better the compression 
ratio and the higher the demands on the CPU when compressing the object. 

By default, all XBEN compression functions will use all available threads on the machine and will
use the highest compression level (9). The XBEN format is only really needed for very large ensemble 
analysis, and machines running such analysis tend to have the compute power to accommodate these
defaults.

### Decompression

Insofar as file decompression goes, what you see is what you get. All of the functions have the 
exact same signature, and should be pretty self-explanatory.

In [7]:
decompress_ben_to_jsonl(
    in_file="example_data/small_example_jsonl_to_ben.jsonl.ben", 
    out_file="example_data/small_example_jsonl_to_ben_to_jsonl.jsonl",
    overwrite=True,
)   

decompress_xben_to_jsonl(
    in_file="example_data/small_example_jsonl_to_xben.jsonl.xben",
    out_file="example_data/small_example_jsonl_to_xben_to_jsonl.jsonl",
    overwrite=True,
)

decompress_xben_to_ben(
    in_file="example_data/small_example_jsonl_to_xben.jsonl.xben",
    out_file="example_data/small_example_jsonl_to_xben_to_ben.jsonl.ben",
    overwrite=True,
)

## PyBEN and GerryChain

As mentioned before, PyBEN was originally designed to work with ensembles generated by programs
like GerryChain, and so we will give a small tutorial here.

> **Note:** in the current version of GerryChain (0.3.2), there are some small peculiarities in
> the way that the `Assignment` class works that require some care.

### Encoding

Working with the PyBEN encoder should feel a lot like working with any python object that handles
writing to files. In particular, we will use the context manager pattern to make sure that the
file is appropriately opened and closed as we write assignment vectors to it.

In [8]:
from gerrychain import Partition, Graph, MarkovChain, updaters, accept
from gerrychain.proposals import recom
from gerrychain.constraints import contiguous
from functools import partial


graph = Graph.from_json("./example_data/gerrymandria.json")

my_updaters = { "population": updaters.Tally("TOTPOP"), }

initial_partition = Partition(
    graph,
    assignment="district",
    updaters=my_updaters
)

ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

proposal = partial(
    recom,
    pop_col="TOTPOP",
    pop_target=ideal_population,
    epsilon=0.01,
    node_repeats=2
)

recom_chain = MarkovChain(
    proposal=proposal,
    constraints=[contiguous],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=10_000
)

Okay, now it is time to write the output into a BEN format. The most important thing that
we need to keep track of here is the order of the `Assignment` returned by GerryChain. In general
GerryChain makes no guarantees about the ordering of the nodes in the output assignment, and to
write to a BEN file, we MUST make sure that the ordering of the values in the assignment vector
lines up with the order of the nodes in the graph.

In [9]:
from pyben import PyBenEncoder

graph_node_order = list(graph.nodes)

with PyBenEncoder("example_data/gerrychain_10000.jsonl.ben", overwrite=True) as encoder:
    for partition in recom_chain.with_progress_bar():
        assignment_series = partition.assignment.to_series()
        # Assignment vectors must be lists of integers
        ordered_assignment = assignment_series.loc[graph_node_order].astype(int).tolist() 
        encoder.write(ordered_assignment)



  0%|          | 0/10000 [00:00<?, ?it/s]

### Decoding

Decoding with PyBEN should also feel fairly simple: just iterate over the file and pull out the 
assignment vector that you would like to work with.

In [10]:
from pyben import PyBenDecoder
import pandas as pd


graph_node_order_series = pd.Index(graph.nodes)

for i, assignment in enumerate(PyBenDecoder("example_data/gerrychain_10000.jsonl.ben")):
    assignment = pd.Series(assignment, index=graph_node_order_series)
    partition = Partition(
        graph,
        assignment=assignment,
        updaters=my_updaters
    )
    print(f"Sample: {i+1}, Cut Edge Count: {len(partition['cut_edges'])}")

Sample: 1, Cut Edge Count: 56
Sample: 2, Cut Edge Count: 50
Sample: 3, Cut Edge Count: 50
Sample: 4, Cut Edge Count: 50
Sample: 5, Cut Edge Count: 44
Sample: 6, Cut Edge Count: 44
Sample: 7, Cut Edge Count: 40
Sample: 8, Cut Edge Count: 40
Sample: 9, Cut Edge Count: 38
Sample: 10, Cut Edge Count: 38
Sample: 11, Cut Edge Count: 38
Sample: 12, Cut Edge Count: 38
Sample: 13, Cut Edge Count: 38
Sample: 14, Cut Edge Count: 38
Sample: 15, Cut Edge Count: 38
Sample: 16, Cut Edge Count: 38
Sample: 17, Cut Edge Count: 38
Sample: 18, Cut Edge Count: 39
Sample: 19, Cut Edge Count: 39
Sample: 20, Cut Edge Count: 39
Sample: 21, Cut Edge Count: 39
Sample: 22, Cut Edge Count: 38
Sample: 23, Cut Edge Count: 40
Sample: 24, Cut Edge Count: 42
Sample: 25, Cut Edge Count: 42
Sample: 26, Cut Edge Count: 42
Sample: 27, Cut Edge Count: 42
Sample: 28, Cut Edge Count: 45
Sample: 29, Cut Edge Count: 45
Sample: 30, Cut Edge Count: 46
Sample: 31, Cut Edge Count: 44
Sample: 32, Cut Edge Count: 44
Sample: 33, Cut E

### Subsampling

Often times, when working with ensembles of plans, it is desirable to subsample from the ensemble
for the sake of winnowing to plans that satisfy some property, and the `PyBenDecoder` has native 
support for all of these things.


We'll work on the data with 100k plans on Colorado Census blocks (there are ~140k blocks in 
Colorado)

In [11]:
decompress_xben_to_ben(
    in_file="example_data/100k_CO_chain.jsonl.xben",
    out_file="example_data/100k_CO_chain.jsonl.ben",
    overwrite=True,
)

In [12]:
for assignment in PyBenDecoder("example_data/100k_CO_chain.jsonl.ben").subsample_indices([1, 23978, 100000]):
    print(assignment[:10])


[8, 8, 5, 4, 5, 5, 5, 5, 3, 3]
[1, 1, 6, 5, 6, 6, 6, 6, 2, 2]
[1, 1, 3, 8, 8, 8, 3, 3, 4, 4]


In [13]:
for assignment in PyBenDecoder("example_data/100k_CO_chain.jsonl.ben").subsample_range(1000,1005):
    print(assignment[:10])

[5, 5, 6, 1, 2, 1, 6, 6, 4, 4]
[5, 5, 1, 1, 2, 1, 1, 1, 4, 4]
[7, 7, 1, 1, 2, 1, 1, 1, 4, 4]
[7, 7, 4, 4, 2, 4, 4, 4, 1, 1]
[7, 7, 4, 4, 2, 4, 4, 4, 1, 1]
[7, 7, 4, 4, 2, 4, 4, 4, 1, 1]


In [14]:
for assignment in PyBenDecoder("example_data/100k_CO_chain.jsonl.ben").subsample_every(10000):
    print(assignment[:10])

[8, 8, 5, 4, 5, 5, 5, 5, 3, 3]
[7, 7, 3, 3, 3, 3, 3, 3, 2, 2]
[5, 5, 1, 3, 4, 4, 1, 1, 3, 3]
[5, 5, 3, 8, 3, 3, 3, 6, 1, 1]
[8, 8, 4, 4, 3, 4, 4, 7, 1, 1]
[5, 1, 7, 3, 3, 7, 7, 3, 6, 6]
[4, 4, 1, 3, 1, 1, 1, 1, 5, 5]
[6, 6, 7, 2, 1, 2, 7, 7, 7, 5]
[4, 4, 8, 1, 8, 8, 8, 3, 7, 7]
[1, 1, 5, 5, 5, 5, 5, 5, 6, 6]
[1, 1, 3, 8, 8, 8, 3, 3, 4, 4]


Of course, you can also do subsampling from XBEN, but the extra compression induces a startup
cost for accessing anything in the file.

In [15]:
for assignment in PyBenDecoder("example_data/100k_CO_chain.jsonl.xben", mode="xben").subsample_indices([1, 23978, 100000]):
    print(assignment[:10])


/tmp/ipykernel_358762/229284435.py:1: UserWarning: XBEN may take a second to start decoding.
  for assignment in PyBenDecoder("example_data/100k_CO_chain.jsonl.xben", mode="xben").subsample_indices([1, 23978, 100000]):


[8, 8, 5, 4, 5, 5, 5, 5, 3, 3]
[1, 1, 6, 5, 6, 6, 6, 6, 2, 2]
[1, 1, 3, 8, 8, 8, 3, 3, 4, 4]


In [16]:
for assignment in PyBenDecoder("example_data/100k_CO_chain.jsonl.xben", mode="xben").subsample_range(1000,1005):
    print(assignment[:10])

/tmp/ipykernel_358762/1010090289.py:1: UserWarning: XBEN may take a second to start decoding.
  for assignment in PyBenDecoder("example_data/100k_CO_chain.jsonl.xben", mode="xben").subsample_range(1000,1005):


[5, 5, 6, 1, 2, 1, 6, 6, 4, 4]
[5, 5, 1, 1, 2, 1, 1, 1, 4, 4]
[7, 7, 1, 1, 2, 1, 1, 1, 4, 4]
[7, 7, 4, 4, 2, 4, 4, 4, 1, 1]
[7, 7, 4, 4, 2, 4, 4, 4, 1, 1]
[7, 7, 4, 4, 2, 4, 4, 4, 1, 1]


In [17]:
for assignment in PyBenDecoder("example_data/100k_CO_chain.jsonl.xben", mode="xben").subsample_every(10000):
    print(assignment[:10])

/tmp/ipykernel_358762/49125867.py:1: UserWarning: XBEN may take a second to start decoding.
  for assignment in PyBenDecoder("example_data/100k_CO_chain.jsonl.xben", mode="xben").subsample_every(10000):


[8, 8, 5, 4, 5, 5, 5, 5, 3, 3]
[7, 7, 3, 3, 3, 3, 3, 3, 2, 2]
[5, 5, 1, 3, 4, 4, 1, 1, 3, 3]
[5, 5, 3, 8, 3, 3, 3, 6, 1, 1]
[8, 8, 4, 4, 3, 4, 4, 7, 1, 1]
[5, 1, 7, 3, 3, 7, 7, 3, 6, 6]
[4, 4, 1, 3, 1, 1, 1, 1, 5, 5]
[6, 6, 7, 2, 1, 2, 7, 7, 7, 5]
[4, 4, 8, 1, 8, 8, 8, 3, 7, 7]
[1, 1, 5, 5, 5, 5, 5, 5, 6, 6]
[1, 1, 3, 8, 8, 8, 3, 3, 4, 4]
